# Análise das tendências políticas no município do Rio de Janeiro
## Por Omar Mesquita

Código do município do Rio de Janeiro: **3304557** (obtido no [site do IBGE](https://www.ibge.gov.br/cidades-e-estados/rj/rio-de-janeiro.html))

[Aqui](https://www.tse.jus.br/servicos-eleitorais/cartorios-e-zonas-eleitorais/pesquisa-a-zonas-eleitorais) pode-se ver as zonas eleitorais no **estado** do Rio de Janeiro. Por uma consulta
SQL podemos visualizar apenas zonas no **município** do Rio: <br><br>

```SQL

    SELECT ano, turno, sigla_uf, id_municipio, zona, numero_partido, resultado, votos
    FROM `basedosdados.br_tse_eleicoes.resultados_candidato_municipio_zona`
    WHERE cargo='presidente'
      AND ano >= 2008
      AND sigla_uf='RJ'
      AND id_municipio = '3304557'
    LIMIT 100
```


In [17]:
# Por favor, execute essa célula antes de continuar
# a leitura para garantir as importações de módulos

import pandas as pd     # Leitura e tratamento de tabelas
import basedosdados as bd   # API para acessar dados do TSE
import os                   # Caminhos e I/O

# Determina o diretório no qual o caderno está sendo executado
currentDir = os.getcwd()

Devido à elevada quantidade de zonas eleitorais no município do Rio de Janeiro, faz-se necessário escolher aquelas que contenham a maior quantidade de pessoas votantes. Essa atitude viabiliza a análise dos dados e maximiza o espaço amostral do estudo *(e?)*

Podemos obter essa informação na página oficial do Tribunal Superior Eleitoral de [estatísticas do eleitorado](https://www.tse.jus.br/eleicoes/estatisticas/estatisticas-de-eleitorado/consulta-por-municipio-zona).

Para esse estudo, foi baixada uma tabela em formato `.csv` contendo dados da última eleição (**outubro de 2022**) no **estado do Rio de Janeiro (RJ)**

**Obs.:** A tabela foi baixada e, em seguida, salva em formato **UTF-8** para evitar problemas de codificação.

In [ ]:
# Armazena o arquivo .csv em um DataFrame
df = pd.read_csv(os.path.join(currentDir,"csv","quantidade_de_eleitores.csv"))

# Cria um DataFrame apenas com as colunas que serão utilizadas (Zona e Quantidade)
# e satisfazem a condição de estarem no município do Rio de Janeiro.
# Em seguida, ordena o DataFrame em ordem decrescente de quantidade de eleitores
# e seleciona as 4 zonas com maior quantidade de eleitores.
df_rj = df[ df["Município"] == "RIO DE JANEIRO"] [['Zona', 'Quantidade']].sort_values(by='Quantidade', ascending=False).head(4)

print(df_rj)

A partir da visualização da tabela, nota-se que as zonas eleitorais **119**, **9**, **179** e **214** são as que concentram maior quantidade de pessoas votantes.

Dessa forma, é possível, agora, fazer uma análise dos resultados eleitorais dessas zonas.

Para tanto, baixou-se uma tabela com informações sobre os resultados das eleições presidenciais de 2022 nas zonas eleitorais previamente citadas. Foi utilizada a API Python do [Base dos Dados](https://basedosdados.org/) e uma consulta SQL no interior da função para se obter tais dados.

In [25]:
# AVISO! Caso você tenha baixado esse caderno ou o repositório,
# não é necessário executar essa célula, pois os dados já foram baixados

# Baixando tabelas correspondentes aos resultados das eleições, gênero e instrução do eleitorado
zonas = [119, 9, 179, 214]

for i in zonas:

    bd.download(
        savepath=os.path.join(currentDir,"csv","resultados",f"resultados_eleicoes_{i}.csv"),
        table_id="resultados_candidato_municipio_zona",
        billing_project_id="ufrjanalytica",
        query= f'''
        SELECT turno, zona, numero_partido, resultado, votos, sigla_partido
        FROM `basedosdados.br_tse_eleicoes.resultados_candidato_municipio_zona`
        WHERE ano = 2022
          AND sigla_uf = "RJ"
          AND id_municipio = "3304557"
          AND cargo = "presidente"
          AND zona = {i}
        LIMIT 1000
        '''
    )

    bd.download(
        savepath=os.path.join(currentDir,"csv","genero",f"genero_eleitorado_{i}.csv"),
        table_id="resultados_candidato_municipio_zona",
        billing_project_id="ufrjanalytica",
        query= f'''
        SELECT genero, eleitores
        FROM `basedosdados.br_tse_eleicoes.perfil_eleitorado_secao`
        WHERE ano = 2022
          AND sigla_uf = "RJ"
          AND id_municipio = "3304557"
          AND zona = {i}
        LIMIT 1000
        '''
    )

    bd.download(
        savepath=os.path.join(currentDir,"csv","instrucao",f"instrucao_eleitorado_{i}.csv"),
        table_id="resultados_candidato_municipio_zona",
        billing_project_id="ufrjanalytica",
        query= f'''
        SELECT instrucao, eleitores
        FROM `basedosdados.br_tse_eleicoes.perfil_eleitorado_secao`
        WHERE ano = 2022
          AND sigla_uf = "RJ"
          AND id_municipio = "3304557"
          AND zona = {i}
        LIMIT 1000
        '''
    )

## Uma breve discussão sobre alinhamento político

Para dar sequência ao estudo, será necessário classificar os partidos com maior expressividade nas zonas eleitorais, isto é, aqueles presentes nas tabelas obtidas pela API do **Base dos Dados**.

Trata-se, no entanto, de um assunto delicado e complexo uma vez que o alinhamento político é um conceito que varia conforme o contexto histórico, socioeconômico e considera outros fatores. Por exemplo, dois partidos distintos podem se posicionar de forma semelhante a respeito de uma dada pauta A, mas terem posições opostas em relação à pauta B.

Assim, para fins de simplificação, será utilizada a classificação política feita pelo [Congresso em Foco](https://congressoemfoco.uol.com.br/area/pais/direita-cresce-e-engole-o-centro-no-congresso-mais-fragmentado-da-historia/), o qual classifica os partidos em esquerda, centro e direita.<sup>1</sup>

Classificando os partidos:

- `PDT (12)`: Esquerda
- `PT (13)`: Esquerda
- `PTB (14)`: Direita
- `MDB (15)`: Centro
- `PSTU (16)`: Esquerda
- `PCB (21)`: Direita
- `PL (22)`: Direita
- `DC (27)`: Direita
- `NOVO (30)`: Direita
- `UNIÃO (44)`: Direita
- `UP (80)`: Esquerda

In [ ]:
# Criação de DataFrames para cada zona eleitoral
df9 = pd.read_csv(os.path.join(currentDir,"csv","resultados","resultados_eleicoes_9.csv"))
df119 = pd.read_csv(os.path.join(currentDir,"csv","resultados","resultados_eleicoes_119.csv"))
df179 = pd.read_csv(os.path.join(currentDir,"csv","resultados","resultados_eleicoes_179.csv"))
df214 = pd.read_csv(os.path.join(currentDir,"csv","resultados","resultados_eleicoes_214.csv"))

# Criação de arrays para cada alinhamento político
esquerda = [12, 13, 16, 80]
direita = [14, 21, 22, 27, 30, 44]
centro = [15]

In [26]:
# Análise da zona eleitoral 9

# Inicializa variáveis para armazenar os votos de cada alinhamento político
esq_9 = 0
dir_9 = 0
centro_9 = 0

# Percorre os arrays de cada alinhamento político e soma os votos de cada partido
for i in esquerda:
    esq_9 += df9.loc[df9["numero_partido"] == i, ["votos"]].values[0][0]

for i in direita:
    dir_9 += df9.loc[df9["numero_partido"] == i, ["votos"]].values[0][0]

for i in centro:
    centro_9 += df9.loc[df9["numero_partido"] == i, ["votos"]].values[0][0]

print("Votos à esquerda na seção 9:", esq_9)
print("Votos à direita na seção 9:", dir_9)
print("Votos ao centro na seção 9:", centro_9)

Votos à esquerda na seção 9: 43137
Votos à direita na seção 9: 62747
Votos ao centro na seção 9: 5695


# Referências bibliográficas

[1] [Congresso em Foco](https://congressoemfoco.uol.com.br/area/pais/direita-cresce-e-engole-o-centro-no-congresso-mais-fragmentado-da-historia/)